In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/capstone24-35/capstone-2024-35/Data Extract/colab

/content/drive/MyDrive/capstone24-35/capstone-2024-35/Data Extract/colab


In [ ]:
!pip install torch transformers datasets
!pip install git+https://github.com/SKTBrain/KoBERT.git@master
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install kobert-transformers
!pip install transformers
!pip install torch
!pip install datasets
!pip install seqeval  # NER 평가를 위한 라이브러리
!pip install transformers[torch] -U

from transformers import BertTokenizerFast, BertForTokenClassification, AdamW
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from kobert_tokenizer import KoBERTTokenizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

# KLUE NER 데이터셋 로드
dataset = load_dataset("klue", "ner")

# 태그 리스트 확인
tag_list = dataset['train'].features['ner_tags'].feature.names
print(tag_list)

# tag2id 및 id2tag 사전 생성
tag2id = {tag: id for id, tag in enumerate(tag_list)}
id2tag = {id: tag for tag, id in tag2id.items()}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O']


In [ ]:
from kobert_transformers import get_kobert_model, get_tokenizer
from transformers import BertForTokenClassification

model_name = "mmoonssun/klue_ner_kobert"
model = BertForTokenClassification.from_pretrained(model_name, num_labels=13)  # num_labels는 데이터셋의 라벨 수에 맞춰 조정
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
import re

def predict_entities(text, model, tokenizer, id2tag):
    # GPU 사용 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 평가 모드로 설정
    model.eval()

    # 입력 문장 토크나이징 및 텐서로 변환
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # 예측 수행
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # 예측 결과에서 가장 높은 확률을 가진 태그 ID를 추출
    predictions = torch.argmax(logits, dim=2)

    # ID를 태그로 변환
    predicted_tags = [id2tag[id.item()] for id in predictions[0]]

    # 토큰화된 텍스트와 예측된 태그 결합
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
    token_tag_pairs = [(token, tag) for token, tag in zip(tokens, predicted_tags) if token not in ["[CLS]", "[SEP]", "[PAD]", "<pad>"]]

    # '▁' 문자를 공백으로 대체하여 보다 자연스러운 출력을 생성
    token_tag_pairs = [(token.replace('▁', ' '), tag) for token, tag in token_tag_pairs]

    return token_tag_pairs

def find_career_status(text):
    # '경력' 다음에 오는 '유', '무', '없', '있' 찾기
    pattern = r'경력\s*:\s*(유|무|없|있)'

    # 문자열에서 패턴에 해당하는 부분 찾기
    match = re.search(pattern, text)

    # 찾은 값을 변수에 저장하고 처리
    if match:
        raw_career = match.group(1)  # 첫 번째 그룹(유|무|없|있)을 추출
        # '없'이나 '있'을 각각 '무', '유'로 변환
        if raw_career == '없':
            career = '무'
        elif raw_career == '있':
            career = '유'
        else:
            career = raw_career

        return career
    else:
        return "경력 유무를 찾을 수 없습니다."

def find_phone_number(text):
    # 정규 표현식으로 전화번호 패턴 찾기
    # 패턴 설명: '010'으로 시작하며, '-'가 있을 수도 있고 없을 수도 있으며, 숫자가 연속으로 나타남
    pattern = r'010-?\d{4}-?\d{4}'

    # 문자열에서 패턴에 해당하는 부분 찾기
    match = re.search(pattern, text)

    # 찾은 전화번호를 변수에 저장하고 출력
    if match:
        phone_number = match.group()
        return phone_number
    else:
        return "전화번호를 찾을 수 없습니다."

In [ ]:
# 예시 문장
text = "김준호 경기도 과천시 ,88살 .전화번호는 010-2093-6627, 경력 : 유"

# 모델을 사용하여 문장에서 개체 추출
predicted_entities = predict_entities(text, model, tokenizer,id2tag)
career = find_career_status(text)
phone_number = find_phone_number(text)
# 결과 출력
print(predicted_entities)
print(career)
print(phone_number)

[(' 김', 'B-PS'), ('준호', 'B-PS'), (' 경기도', 'B-LC'), (' ', 'B-LC'), ('과', 'B-LC'), ('천', 'B-LC'), ('시', 'B-LC'), (' ', 'B-LC'), (',', 'O'), ('88', 'B-QT'), ('살', 'O'), (' ', 'O'), ('.', 'O'), ('전', 'O'), ('화', 'O'), ('번호', 'O'), ('는', 'O'), (' 0', 'O'), ('10', 'O'), ('-20', 'O'), ('93', 'O'), ('-', 'O'), ('66', 'O'), ('27', 'O'), (',', 'O'), (' 경력', 'O'), (' :', 'O'), (' 유', 'O')]
유
010-2093-6627


In [ ]:
# 개체 유형별로 정보를 추출하고 결합하는 함수 정의
def extract_and_combine_entities(predicted_entities):
    name = ""
    location = ""
    age = ""

    for token, tag in predicted_entities:
        if tag == 'B-PS':  # 이름 추출
            name += token.strip()
        elif tag == 'B-LC':  # 위치 추출
            if token == " ":
              location += token
            else:
              location += token.strip()
        elif tag == 'B-QT':  # 나이(수량) 추출
            age = token.strip()

    # 결과 반환
    return {"name": name, "location": location, "age": age}

# 함수를 사용하여 변수에 저장
entities_combined = extract_and_combine_entities(predicted_entities)

# 결과 출력
entities_combined["career"] = career
entities_combined["phone_number"] = phone_number
print(entities_combined)

{'name': '김준호', 'location': '경기도 과천시 ', 'age': '88', 'career': '유', 'phone_number': '010-2093-6627'}


In [ ]:
pip install pymongo

In [ ]:
from pymongo import MongoClient

# MongoDB 서버에 연결. 여기서는 로컬 MongoDB 인스턴스를 사용합니다.
client = MongoClient('mongodb://localhost:27017/')

# 'mydatabase'라는 데이터베이스 선택. 없다면 새로 생성됩니다.
db = client['mydatabase']

# 'people'이라는 컬렉션 선택. 없다면 새로 생성됩니다.
collection = db['people']

# 딕셔너리를 MongoDB 컬렉션에 삽입
result = collection.insert_one(entities_combined)

# 삽입된 문서의 ID 출력
print("Inserted document ID:", result.inserted_id)